In [1]:
from os import environ
from pathlib import Path
from json import loads, dumps
from random import choice
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10, temperature = 1):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query, temperature)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [2]:
def giveStatement(topic):
    query = f'Give me a short statement about the topic "{topic}". Return the answer quoted in "". Return nothing but the statement.'
    def answerConversion(answer):
        assert answer.startswith('"') and answer.endswith('"')
        return answer[1:-1]
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [3]:
statement = giveStatement("Mass")
print(statement)

Mass is a fundamental property of physical objects that quantifies the amount of matter they contain; it is a central concept in physics that influences gravity and inertia.


In [6]:
def getPartialStatements(statement):
    query = f'What are the partial statements that are contained in the statement "{statement}" and stand by them self? Return all partial statements as strings that do not depend on each other in a list formatted as ["Partial Statement 1", "Partial Statement 2", ...]. If there are no partial statements the list should only contain the original statement. Return nothing but the list.'
    return tryRecieveAnswer(query, answerConversion = listAnswerConversion)[0]

In [8]:
partialStatements = getPartialStatements(statement)
print(partialStatements)

['Mass is a fundamental property of physical objects that quantifies the amount of matter they contain.', 'Mass is a central concept in physics that influences gravity and inertia.']


In [18]:
def findReasoningStep(statement):
    query = f'The statement "Earth orbits a star" can be concluded from the two statements "Earth is a planet" and "Planets orbit stars". Give me a list of statements from which the statement "{statement}" can be concluded. All statements in the list should stand by them selfs. Return the list of statements as strings formatted as ["Statement 1", "Statement 2", ...]. Return nothing but the list.'
    return tryRecieveAnswer(query, answerConversion = listAnswerConversion)[0]

In [19]:
findReasoningStep("A pendulum swings faster when it is shorter")

['The period of a pendulum is determined by its length',
 'A shorter length results in a shorter period',
 'A shorter period means faster swings']

In [20]:
def findListOfAxiomsToAnswerQuestion(question):
    query = f'What are the axioms that are needed to answer the question "{question}"? Return the list of statements as strings formatted as ["Axiom 1", "Axiom 2", ...]. Return nothing but the list.'
    return tryRecieveAnswer(query, answerConversion = listAnswerConversion)[0]

In [22]:
findListOfAxiomsToAnswerQuestion("How can a rocket be launched into an orbit around the Earth?")

['A rocket must possess a propulsion system to counteract gravitational forces and achieve the necessary altitude and velocity.',
 "Newton's laws of motion apply, especially the principle that for every action, there is an equal and opposite reaction, essential for a rocket’s propulsion.",
 "Knowledge of Earth's gravitational field is crucial, to calculate the necessary escape velocity and to establish a stable orbit.",
 'Orbital mechanics principles must be followed for determining the right trajectory and velocity for the rocket to achieve stable orbit around the Earth.',
 'The rocket must have a structure capable of withstanding the stresses of launch, including thermal and mechanical stresses.',
 "Fuel and mass calculations are essential for determining how much fuel is needed and managing the rocket's weight for achieving the desired orbit."]

In [ ]:
def get_partial_statements(statement):
    query = f'What are the partial statements that are contained in the statement "{statement}" and stand by themselves? Return all partial statements as strings that do not depend on each other in a list formatted as ["Partial Statement 1", "Partial Statement 2", ...]. If there are no partial statements, the list should only contain the original statement. Return nothing but the list.'
    return tryRecieveAnswer(query, answerConversion=listAnswerConversion)[0]
